# Vika 8

Dagsetning: 5. apríl

## Sýnidæmi úr bók bls. 253

Olíunotkun á heimsvísu milli 1980 og 2013.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Gögn lesin úr skrá
data = np.genfromtxt('oliunotkun.csv', skip_header=True, delimiter='\t')
x = data[:,0] - 1980
y = data[:,1]/1000 # Milljónir tunna

# Finna líkan á forminu f(t)=θ1 + θ2*x
m = x.shape[0]
A = np.c_[np.ones(m), x]
theta,res,rank,_ = np.linalg.lstsq(A, y, rcond=False)

# Teikna gögn ásamt bestu línu
plt.scatter(x + 1980, y)
yp = np.dot(np.c_[np.ones(m), x], theta) # Reiknum spágildi fyrir punktana í x
plt.plot(x + 1980, yp, 'r-')
plt.xlabel('Ár')
plt.ylabel('Olíunotkun (milljónir tunna á dag)')
plt.title('Olínotkun í heiminum á árunum 1980 - 2013')
plt.show()

# Drögum línuna frá gögnunum til að sjá sveiflur betur
plt.plot(x + 1980, y - yp, 'bo-')
plt.xlabel('Ár')
plt.ylabel('Olíunotkun (milljónir tunna á dag)')
plt.title('Olínotkun eftir að línuleg leitni hefur verið fjarlægð')
plt.show()

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 0: ordinal not in range(128)

## Aðhvarfsgreining með margliðum

Notum tilbúin gögn að þessu sinni. Við þekkjum því undirliggjandi fallið $f$ og við ætlum að sjá hversu vel okkur gengur að nálga það í framhaldinu. Gildin á $x$ eru slembin og $y_i=0.5 x_i^4 - 0.4x_i^3 - 4x_i^2 + x_i + 2 + \epsilon_i$ þar sem $\epsilon_i$ er (normaldreift) "suð". 

In [ ]:
def tilbuin_gogn(N):
    # Tilbúið gagnasafn
    x = np.random.uniform(-3,3,N)
    y = 0.5*x**4 -0.4*x**3 - 4*x**2+x + 2 + np.random.normal(0.0,0.5,N) # 4. stigs margliða
    return x,y

In [ ]:
N=20
x,y = tilbuin_gogn(N)

for k in [1,2,4,8,16]:
    pz = np.polyfit(x,y,k) # finnum k-ta stigs margliðu
    p = np.poly1d(pz) # Klasi sem einfaldar umstang með margliður
    xp = np.linspace(min(x),max(x),100)
    plt.plot(x,y,'.',xp,p(xp),'-') # plottum gögn vs margliðu
    print(p)
    plt.show()

Þegar gráða margliðu hækkar þá lækkar ferviksskekkja en það merkir ekki endilega að líkanið spái réttilega fyrir um óséð gögn, þ.e. gögn sem voru ekki notuð til að meta stika líkansins. Setjum til hliðar hluta gagna ("prófunargögn"), t.d. 20% og notum RMS skekkju á þau gögn sem mælikvarða á gæði líkans.

In [ ]:
# Setja 
ntrain=int(0.2*N)
x1 = x[:-ntrain] # Þjálfunargögn
y1 = y[:-ntrain]
x2 = x[-ntrain:] # Prófunargögn
y2 = y[-ntrain:]
print(x1)

def rms(u):
    return np.linalg.norm(u)/np.sqrt(len(u))

In [ ]:
test_rms = []
train_rms = []
for k in range(15):
    pz = np.polyfit(x1,y1,k) # finnum k-ta stigs margliðu
    p = np.poly1d(pz)
    train_rms.append(rms(y1 - p(x1))) # RMS skekkja þjálfunargagna
    test_rms.append(rms(y2 - p(x2))) # RMS skekkja prófunargagna
print(test_rms)


In [ ]:
train_rms

In [ ]:
test_rms

In [ ]:
plt.plot(range(len(test_rms)), train_rms,'-',color='blue',label='þjálfunargögn')
plt.plot(range(len(train_rms)), np.clip(test_rms,0,4),'-',color='red', label='prófunargögn')
plt.legend()
plt.show()

Af grafinu sést að margliða með gráðu á bilinu 4 til 8 gefur lægsta spáskekkju (getur breyst lítillega ef kóðinn er keyrður aftur því gögn eru slembin).

## Cross-validation bls. 264-266

Sýnidæmi úr bók um aðhvarfsgreiningarlíkan fyrir húsnæðisverð (sjá líka bls. 39). Spálíkanið er $\hat{y} = x^T \beta + v = \beta_1 x_1 + \beta_2 x_2 + v$.

<img src="vika8regression.png" width="400" height="400" />

## Minnstu kvaðrata flokkari

Sýnidæmi bls. 289 - 290 (Iris gagnasafnið)

<img src="vika8iris.png" width="400" height="400" />

In [ ]:
data=np.genfromtxt('iris.csv',skip_header=True)

X=data[:,0:3] # Sepal (bikarblað) length, Sepal width, Petal (krónublað) length, Petal Width
y=data[:,-1] # Flokkar Setosa=1, Versicolor=2, Virginica=3

# Breyta í 2ja flokka verkefni, Virginica eða ekki
y[y==3]=-1
y[y==2]=1

N = X.shape[0]
A = np.c_[np.ones(N), X]
theta,res,rank,_ = np.linalg.lstsq(A, y, rcond=False)
print("theta:", theta) # Ekki sömu gildi og í bók

# Spá
ypred=np.sign(np.dot(A,theta))

print(np.sum(y != ypred)/N)

# Flokkun á handskrifuðum stöfum (MNIST gagnasafnið) bls. 302 - 304

<img src="vika8mnist.png" width="600" height="600" />

U.þ.b. 14% skekkja á prófunar- og þjálfunargögn þ.a. nákvæmni á óséð gögn er líklega svipuð.

Skekkja í flokkun hjá mönnum er u.þ.b. 2%.

Bestu flokkarar ná minna en 1% skekkju!